### 인공 신경망 최적화 기법

In [8]:
import torch.optim as optim
import torchvision

In [9]:
model = torchvision.models.resnet18(weights=None) # weights: pretrained model을 결정

#### 1. SGD (Stochasic Gradient Descent)
기본이 되는 최적화 기법이다.

- 하나의 **데이터 샘플(또는 미니 배치)** 만 가지고 손실 함수의 기울기(gradient)를 계산해서 파라미터를 업데이트 한다.
- 전체 데이터를 한 번 다 보고 나서 업데이트하는 **GD(Gradient Descent)** 보다 훨씬 빠르고 메모리 효율적이다.
- 단순하고 계산 비용이 낮다.
- 하지만 진동이 크고, 지역 최솟값에 빠질 수 있다.
- 그래서 아래의 Momentum 같은 기능을 더 한다.

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

#### 2. SGD with Momentum
- 관성을 줘서 이전 업데이트 방향을 기억하고, 같은 방향이면 더 빠르게, 다른 방향이면 천천히 움직이게 한다.
- SGD보다 훨씬 빠르게 수렴
- 진동을 줄이고, local minimum에서 빠져나올 확률도 높다.

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#### 3. Adam (Adaptive Moment Estimation)
자동으로 학습률을 조정 + 모멘텀

- 학습률과 모멘텀 조정을 동시에 해주는 효과적인 방법
- (기울기의 평균) 갑자기 방향을 바꾸어도 이전 방향을 기억해서 부드럽게 이동하게 한다.
- (기울기의 제곱의 평균) 파라미터 마다 학습률이 자동으로 조정 (기울기 값이 큰 파라미터는 학습률을 작게, 작은 파라미터는 학습률을 크게 조절해 준다.)
- 대부분의 문제에서 튜닝 없이도 잘 작동
- 이미지, 텍스트, 음성 등 거의 모든 딥러닝 분야에서 인기가 많음
- 하지만 너무 자동이라 가끔 과적합이나 일반화 성능이 떨어질 수 있음

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

#### 4. (Learning Rate) Scheduling
시간이 지나면 학습률을 점점 줄여서 더 섬세하게 조정한다.
- 초반에는 빠르게 배우고(큰 학습률 적용), 후반에는 천천히 섬세하게 조정(학습률을 점점 감소)한다.
- 초기에는 큰 학습률로 전역 최적점(global optimum)을 빠르게 탐색하고, 후반에는 학습률을 줄여 미세 조정(fine-tuning)함으로써 안정적인 수렴과 일반화 성능을 동시에 확보할 수 있다.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1) # 30 epoch마다 학습률을 10%로 줄여서 점점 섬세하게 학습하게 만든다.

```python
... 중략 ...

for epoch in range(400): 
    running_loss = 0.0
    
    for data in trainloader: 
        inputs, values = data 
        optimizer.zero_grad() 
        outputs = model(inputs) 
        loss = criterion(outputs, values) 
        loss.backward() 
        optimizer.step() 
        
        ... 중략 ...
    
    scheduler.step() # 스케줄링을 통한 학습률 조정 (일반적으로 배치 계산 밖에서 적용)
```